100%|██████████| 2231142/2231142 [00:52<00:00, 42467.49it/s]


## Trening
Import bibliotek i procedura normalizacyjna

In [1]:
#import prodigy
import spacy
import csv
import os

from pathlib import Path
#from prodigy.models.ner import EntityRecognizer

nlp = spacy.load("en_core_web_sm")
#name_dict, desc_dict = load_food_entities()
#kb = spacy.kb.KnowledgeBase(vocab=nlp.vocab, entity_vector_length=300)

import re
import itertools
from itertools import chain, combinations
import warnings
warnings. filterwarnings('ignore')
stopwords = nlp.Defaults.stop_words
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
def normalize_name(name):
    name = re.sub(r'[^a-zA-Z]', ' ', name)
    name = re.sub(r'\s+', ' ', name)
    name = name.lower()
    name = ' '.join([token for token in name.split(' ') if token not in stopwords])
    stemmed_name = " ".join([ps.stem(token.text) for token in nlp(name)])
    return stemmed_name

Wczytanie zbioru danych

In [2]:
import pandas as pd
df = pd.read_csv("recipeNLG_with_entities.csv")

Ekstrakcja z JSON potrzebnych pól (wykryte NER o typie `FOOD`)

In [3]:
import json
from tqdm import tqdm

corpus = []
json_fail_count = 0
json_fail = []
for ingredients_entities in tqdm(df["ingredients_entities"].tolist()):
    sentence = []
    try:
        for entity in json.loads(ingredients_entities):
            if entity["type"] == "FOOD":
                sentence.append("_".join(normalize_name(entity["entity"]).split(" ")))
        corpus.append(" ".join(sentence).lower())
    except:
        json_fail.append(ingredients_entities)
        json_fail_count += 1

100%|██████████| 2231142/2231142 [22:45:56<00:00, 27.22it/s]   


Zapis do pliku wyesktrachowanej reprezentacji do treningu

In [4]:
with open('dataset_new_forfasttext_lower.csv', 'w') as csvfile:
    for i in corpus:
        csvfile.write(i.lower())
        csvfile.write("\n")

Trening modelu, zapis wyniku

In [25]:
import fasttext
model = fasttext.train_unsupervised('dataset_new_forfasttext_instructions.csv',thread=8, minn=4, maxn=6, dim=50, verbose=0)

In [26]:
model.save_model("dataset_new_instructions.bin")

In [7]:
len(model.words)

32781

## Wczytanie modelu

In [1]:
import fasttext
model = fasttext.load_model("dataset_new.bin")
model.get_nearest_neighbors('chicken')

[(0.9290194511413574, 'citi_chicken'),
 (0.893699049949646, 'mix_chicken_piec'),
 (0.8879286646842957, 'chicken_halv'),
 (0.8790009021759033, 'chicken_portion'),
 (0.8712169528007507, 'broil_chicken'),
 (0.8696103692054749, 'chicken_chicken'),
 (0.867918074131012, 'chicken_cut'),
 (0.8649100661277771, 'chicken_ala_king'),
 (0.8585206866264343, 'meat_chicken'),
 (0.8568282723426819, 'veggi_chicken')]

Proste wyszukiwanie czy składniki znajdują się w modelu

In [13]:
[i for i in model.words if 'gouda' in i ]

['gouda_chees',
 'gouda',
 'shred_gouda_chees',
 'chees_gouda',
 'goat_gouda',
 'boar_s_head_chipotl_gouda']

Rekomendacja

In [18]:
for test_words in ['chicken', 'tofu', 'milk', 'egg', 'banana', 'lentil',  'soy_cream', 'butter', 'gouda']:
    print(test_words)
    for i in model.get_nearest_neighbors(test_words):
        print(i)

chicken
(0.9290194511413574, 'citi_chicken')
(0.893699049949646, 'mix_chicken_piec')
(0.8879286646842957, 'chicken_halv')
(0.8790009021759033, 'chicken_portion')
(0.8712169528007507, 'broil_chicken')
(0.8696103692054749, 'chicken_chicken')
(0.867918074131012, 'chicken_cut')
(0.8649100661277771, 'chicken_ala_king')
(0.8585206866264343, 'meat_chicken')
(0.8568282723426819, 'veggi_chicken')
tofu
(0.9260441660881042, 'tofu_silken')
(0.8897832632064819, 'bake_tofu')
(0.8827080726623535, 'egg_tofu')
(0.8749420046806335, 'tofu_firm')
(0.870351254940033, 'silk_tofu')
(0.8676044940948486, 'silki_tofu')
(0.8665119409561157, 'tofu_skin')
(0.8615848422050476, 'silken_tofu_firm')
(0.8584445118904114, 'silken_tofu')
(0.8566809296607971, 'silken')
milk
(0.8302230834960938, 'evapor_milk')
(0.8208866119384766, 'butter')
(0.8154518008232117, 'egg')
(0.8121494650840759, 'milnot_cream')
(0.8046228885650635, 'milnot_milk')
(0.803139865398407, 'dairi_half_half')
(0.7996000647544861, 'evapor')
(0.79781383275

## Eksport zanurzeń

In [9]:
with open("embeddings.txt", 'w') as f:
    for i in model.get_words():
        f.write(i)
        
        for j in model.get_word_vector(i):
            f.write(",")
            f.write(str(j))
        f.write("\n")

In [12]:
df["directions"][0]

'In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper. Let stand until firm, about 30 minutes.'

# Dodatek: tworzenie pliku do treningu modelu na tekstach całych poleceń 

Założenie: zbiór danych został już wczytany tak jak w kodzie wyżej

In [19]:
with open('dataset_new_forfasttext_instructions.csv', 'w') as csvfile:
    for i in tqdm(df["directions"]):
        if type(i) !=  str:
            print("ERR",i)
            continue
        csvfile.write(i.lower())
        csvfile.write("\n")

 81%|████████  | 1809148/2231142 [00:03<00:01, 421758.64it/s]

ERR nan


100%|██████████| 2231142/2231142 [00:04<00:00, 509885.51it/s]


In [23]:
df.iloc[1809149,]

title                                                      Fancy Meatloaf
ingredients             1/2 loaf Italian bread, crust removed, torn in...
directions              Preheat the oven to 375 degrees. Soak the brea...
link                                  cooking.nytimes.com/recipes/1012686
source                                                          Recipes1M
NER                     ["Italian bread", "milk", "ground beef", "grou...
ingredients_entities    [{"start": 0, "end": 3, "type": "QUANTITY", "e...
Name: 1809149, dtype: object